In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai import *
test1=pd.read_csv("../input/inputdatann/test.csv",index_col="Id")


In [ ]:
type(test1)

In [ ]:
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


In [ ]:
df=pd.read_csv("../input/inputdatann/train.csv",index_col="Id")

In [ ]:
df.head()

In [ ]:
df["start"]=df["Timestep"].str.slice(2,6)




In [ ]:
test1["start"]=test1["Timestep"].str.slice(2,6)

In [ ]:
test1["stop"]=test1["Timestep"].str.slice(9,13)

In [ ]:
df["stop"]=df["Timestep"].str.slice(9,13)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
test1.head()

In [ ]:
del df["Timestep"]

In [ ]:
del test1["Timestep"]

In [ ]:
df["start"]=df["start"].astype(int)
test1["stop"]=test1["stop"].astype(int)
test1["start"]=test1["start"].astype(int)
df["stop"]=df["stop"].astype(int)

In [ ]:
dfg=pd.get_dummies(df)

In [ ]:
y_train=dfg["Expected"]

In [ ]:
dfg2=dfg.copy()

In [ ]:
del dfg2["Expected"]

In [ ]:
X_train=dfg2

In [ ]:
combine=pd.concat([df,test1])

In [ ]:
type(df["start"][0])

In [ ]:
cond=combine.start<1990

In [ ]:
df.tail()

In [ ]:
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]

In [ ]:
splits = (list(train_idx),list(valid_idx))

In [ ]:
procs = [Categorify, FillMissing]

In [ ]:
dep_var="Expected"

In [ ]:
cont,cat = cont_cat_split(combine, 1, dep_var=dep_var)
to = TabularPandas(combine, procs, cat, cont, y_names=dep_var, splits=splits)

In [ ]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

In [ ]:
m = RandomForestRegressor(n_estimators=200)


In [ ]:
m.fit(X_train,y_train)

In [ ]:
m.predict(pd.get_dummies(test1))

In [ ]:
rf=m.predict(valid_xs)

In [ ]:
rf

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=1)

In [ ]:
neigh.fit(xs, y)

In [ ]:
knn=neigh.predict(valid_xs)

In [ ]:
knn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=10, include_bias=False)

In [ ]:
poly.fit(xs)
xs_poly=poly.transform(xs)

In [ ]:
reg=LinearRegression().fit(xs_poly, y)

In [ ]:
lr=reg.predict(poly.transform(valid_xs))

In [ ]:
lr

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
regr = AdaBoostRegressor(random_state=0, n_estimators=100)

In [ ]:
regr.fit(xs,y)

In [ ]:
ada=regr.predict(valid_xs)

In [ ]:
ada

In [ ]:
submission_en=pd.DataFrame()

In [ ]:
ids=np.arange(len(test1))

In [ ]:
preds=(0.8*rf)+(0.2*knn)

In [ ]:
submission_en["Id"]=ids
submission_en["Predicted"]=preds

In [ ]:
submission_en.set_index("Id")

In [ ]:
submission_en.to_csv("en.csv",index=False)